In [5]:
from flask import Flask, request, jsonify
from werkzeug.security import generate_password_hash

# Step 1: Create an instance of the Flask class
app = Flask(__name__)

# Step 2: Define your routes
@app.route('/register', methods=['POST'])
def register():
    data = request.get_json()
    password_hash = generate_password_hash(data['password'])
    
    # Example: Save the user to the database
    # For now, we'll just return the hashed password as a demonstration
    return jsonify({"hashed_password": password_hash})

# Step 3: Run the app
if __name__ == '__main__':
    app.run(debug=True, use_reloader=False)


 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [03/Sep/2024 19:23:07] "POST /register HTTP/1.1" 200 -
127.0.0.1 - - [03/Sep/2024 19:28:12] "POST /register HTTP/1.1" 200 -
127.0.0.1 - - [03/Sep/2024 19:34:43] "GET /register HTTP/1.1" 405 -


In [23]:
from flask import Flask, request, jsonify
import mysql.connector
from mysql.connector import Error

app = Flask(__name__)

# Configure MySQL connection
def get_db_connection():
    return mysql.connector.connect(
        host='DESKTOP-K62B7PD',
        user='shashank-02',
        password='Shashank@02',
        database='railway_db'
    )

# Define POST route
@app.route('/api/endpoint', methods=['POST'])
def insert_data():
    if request.is_json:
        data = request.get_json()
        field1 = data.get('field1')
        field2 = data.get('field2')

        try:
            connection = get_db_connection()
            cursor = connection.cursor()
            query = 'INSERT INTO your_table (field1, field2) VALUES (%s, %s)'
            cursor.execute(query, (field1, field2))
            connection.commit()
            cursor.close()
            connection.close()
            return jsonify({'message': 'Data inserted successfully'}), 200
        except Error as e:
            return jsonify({'error': str(e)}), 500
    else:
        return jsonify({'error': 'Request must be JSON'}), 400

if __name__ == '__main__':
    app.run(port=5001, debug=True, use_reloader=False)



 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5001
Press CTRL+C to quit
127.0.0.1 - - [03/Sep/2024 19:58:37] "GET / HTTP/1.1" 404 -
127.0.0.1 - - [03/Sep/2024 19:58:37] "GET /favicon.ico HTTP/1.1" 404 -


In [ ]:
@app.route('/register', methods=['POST'])
def register_user():
    data = request.get_json()
    password_hash = generate_password_hash(data['password'])

    query = "INSERT INTO users (username, password_hash) VALUES (%s, %s)"
    values = (data['username'], password_hash)
    
    cursor.execute(query, values)
    db.commit()

    return jsonify({"message": "User registered successfully!"}), 201


In [9]:
@app.route('/login', methods=['POST'])
def login():
    data = request.get_json()
    
    query = "SELECT * FROM users WHERE username = %s"
    cursor.execute(query, (data['username'],))
    user = cursor.fetchone()

    if user and check_password_hash(user['password_hash'], data['password']):
        access_token = create_access_token(identity=user['id'])
        return jsonify(access_token=access_token), 200

    return jsonify({"message": "Invalid credentials!"}), 401


In [14]:
@app.route('/admin/add-train', methods=['POST'])
def addtrain():
    api_key = request.headers.get('X-API-KEY')
    if api_key != 'your_admin_api_key':
        return jsonify({"message": "Unauthorized access!"}), 403

    data = request.get_json()
    
    query = "INSERT INTO trains (name, source, destination, total_seats, available_seats) VALUES (%s, %s, %s, %s, %s)"
    values = (data['name'], data['source'], data['destination'], data['total_seats'], data['total_seats'])
    
    cursor.execute(query, values)
    db.commit()

    return jsonify({"message": "Train added successfully!"}), 201


In [16]:
@app.route('/trains/availability', methods=['GET'])
def getavailability():
    source = request.args.get('source')
    destination = request.args.get('destination')

    query = "SELECT id, name, available_seats FROM trains WHERE source = %s AND destination = %s"
    cursor.execute(query, (source, destination))
    trains = cursor.fetchall()

    results = [
        {
            "train_id": train['id'],
            "train_name": train['name'],
            "available_seats": train['available_seats']
        } for train in trains
    ]
    return jsonify(results), 200


In [18]:
@app.route('/book-seat', methods=['POST'])
def book_seat():
    user_id = get_jwt_identity()
    data = request.get_json()
    train_id = data['train_id']

    try:
        query = "START TRANSACTION"
        cursor.execute(query)

        query = "SELECT available_seats FROM trains WHERE id = %s FOR UPDATE"
        cursor.execute(query, (train_id,))
        train = cursor.fetchone()

        if train and train['available_seats'] > 0:
            new_seat = train['available_seats']
            
            query = "UPDATE trains SET available_seats = available_seats - 1 WHERE id = %s"
            cursor.execute(query, (train_id,))
            
            query = "INSERT INTO bookings (user_id, train_id, seat_number) VALUES (%s, %s, %s)"
            cursor.execute(query, (user_id, train_id, new_seat))
            
            db.commit()
            return jsonify({"message": "Seat booked successfully!"}), 200
        else:
            db.rollback()
            return jsonify({"message": "No seats available!"}), 400

    except Exception as e:
        db.rollback()
        return jsonify({"message": "An error occurred while booking the seat."}), 500


In [19]:
@app.route('/booking/<int:booking_id>', methods=['GET'])
def get_booking(booking_id):
    user_id = get_jwt_identity()
    
    query = "SELECT * FROM bookings WHERE id = %s AND user_id = %s"
    cursor.execute(query, (booking_id, user_id))
    booking = cursor.fetchone()

    if booking:
        return jsonify({
            "booking_id": booking['id'],
            "train_id": booking['train_id'],
            "seat_number": booking['seat_number'],
            "booking_time": booking['booking_time']
        }), 200
    return jsonify({"message": "Booking not found!"}), 404
